In [2]:
import os
# Change this if you don't want the data to be extracted in the current directory.
data_dir = '.'
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

In [3]:
import os
import numpy as np
import pickle
import pandas as pd
import json
import re
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Audio
from IPython.display import display
from scipy.io import wavfile
from itertools import chain 
import string
import Levenshtein as Lev
from itertools import groupby
import scipy.stats as st
from scipy import signal
import nltk
from scipy.stats import wasserstein_distance as wd
from sklearn.metrics.pairwise import euclidean_distances as ed
from pyemd import emd
from hyphenate import hyphenate_word
from itertools import islice 
from collections import OrderedDict
import csv

In [4]:
#####################################################################################
########### set the Data files paths on Conda Notebook on Ubuntu #################### 

### 
os.chdir('/home/mmm2050/QU_DFKI_Thesis/Experimentation/ASR_Accent_Analysis_De')
main_path=os.getcwd()

if not os.path.exists(main_path+'/Data_results'):
  os.makedirs(main_path+'/Data_results')

if not os.path.exists(main_path+'/Figures_results'):
  os.makedirs(main_path+'/Figures_results')

Data_path=main_path+'/Data/'
Data_results_path=main_path+'/Data_results/'
json_file_path=Data_path+'results.json'
json_file_path_small=Data_path+'results_small.json'
validated_tsv_path=Data_path+'validated.tsv'
validated_tsv_path_small=Data_path+'validated_small.tsv'
 

validated_en_tsv_path=main_path+'/DeepSpeech/data/validated.tsv'
validated_en_tsv_path_small=main_path+'/DeepSpeech/data/validated_small.tsv'

 
os.getcwd()

'/home/mmm2050/QU_DFKI_Thesis/Experimentation/ASR_Accent_Analysis_De'

### <font color=white> **Creating a `En` dictionary `file_meta` for the Audio files , Accents** </font> 


In [12]:
#@title Creat a EN dictionary for the Audio files , Accents#######
###################################################################

# initialize an empty dictionary
file_meta = {}

# read the TSV file
with open(validated_en_tsv_path_small, 'r') as f:
    # skip the header row
    next(f)
    # iterate over the remaining rows
    for line in f:
        # split the line into columns
        cols = line.strip().split('\t')
        # extract the relevant columns
        filename = cols[1].split('.')[0]
        accent = cols[7]
        transcript = cols[2]
        # create a dictionary for this file
        file_dict = {'accent': accent, 'transcript': transcript}
        # add the dictionary to the file_meta dictionary
        file_meta[filename] = file_dict
        

        # Open a new CSV file in write mode
with open(main_path+'/DeepSpeech/data/file_meta.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(file_meta.keys())

    # Write the values row
    writer.writerow(file_meta.values())

In [13]:
file_meta["common_voice_en_533247"]

{'accent': 'australia',
 'transcript': 'The burning fire had been extinguished.'}

In [15]:
file_meta["common_voice_en_533247"]['accent']

'australia'

In [16]:
file_meta["common_voice_en_533247"]['transcript']

'The burning fire had been extinguished.'

### <font color=white> **Creating a `De` dictionary `file_meta` for the Audio files , Accents** </font> 


In [18]:
#@title Creat a De dictionary for the Audio files , Accents#######
###################################################################

# initialize an empty dictionary
file_meta = {}

# read the TSV file
# with open(validated_tsv_path_small, 'r') as f:
with open(validated_tsv_path_small, 'r') as f:
    # skip the header row
    next(f)
    # iterate over the remaining rows
    for line in f:
        # split the line into columns
        cols = line.strip().split('\t')
        # extract the relevant columns
        filename = cols[1].split('.')[0]
        accent = cols[7]
        transcript = cols[2]
        # create a dictionary for this file
        file_dict = {'accent': accent, 'transcript': transcript}
        # add the dictionary to the file_meta dictionary
        file_meta[filename] = file_dict
        

        # Open a new CSV file in write mode
with open(main_path+'/DeepSpeech/data/file_meta.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(file_meta.keys())

    # Write the values row
    writer.writerow(file_meta.values())

In [19]:
file_meta["common_voice_de_30676740"]

{'accent': 'Österreichisches Deutsch',
 'transcript': 'Gerschler wurde von seinen Großeltern und einer Tante erzogen.'}

In [20]:
file_meta["common_voice_de_30676740"]['accent']

'Österreichisches Deutsch'

In [21]:
file_meta["common_voice_de_30676740"]['transcript']

'Gerschler wurde von seinen Großeltern und einer Tante erzogen.'

### **Reading `Phones` and `End_times` from the `Textgrid` of multiple file**:

In [22]:
import textgrid
import csv
import os
import glob

#@title  Process "align.json" files contents and import end_times,phones to pass it to ###
		################################## file_meta dict ########################################
		##########################################################################################
		 

'''
##########################################################################################
##### Input : the output align.json of an Audio (From Montreal Forced Aligner Output)
##### Output:
##########################################################################################
'''

# dir_path = os.path.dirname(os.path.abspath(__file__))
############### # in this case should the textgrid ile located at the same working file
############### AttributionAnalysis_De_NB18042023.ipynb

Lan ='De'

if Lan =='De':
    folder_path = main_path+'/DeepSpeech/data/textgrid/' # replace with the actual path of your directory
elif Lan=='En':
    folder_path = main_path+'/DeepSpeech/data/textgrid_En/' # replace with the actual path of your directory

extension = '.TextGrid'

# Get a list of all files with the specified extension in the folder
files = glob.glob(os.path.join(folder_path, '*' + extension))

# Loop through the files and print their names
for audio_file_textgrid in files:
    # print(os.path.basename(file))
    file_name=os.path.basename(audio_file_textgrid)
# dir_path =main_path+'/DeepSpeech/data/textgrid/'
# file_name = "common_voice_de_30676740"
    # file_path = dir_path + "/" + file_name


    # tg = textgrid.TextGrid.fromFile(dir_path + "/" + file_name + ".TextGrid")
    tg = textgrid.TextGrid.fromFile(folder_path + "/" + file_name)
    csv_input = [[], []]


    print("------- IntervalTier Example -------")
    print(os.path.basename(audio_file_textgrid),'- export Phnes and End_times Done')
    file_name_id=os.path.splitext(os.path.basename(audio_file_textgrid))[0]
    t = tg.tiers
    for i in range (len(tg.getNames())): 
        if (tg[i].name == "phones"): 
            # print(tg[i].name)
            for j in range (len(tg[i].intervals)):
                # print(tg[i][j].mark)
                csv_input[0].append(tg[i][j].mark)
                csv_input[1].append(tg[i][j].maxTime)

        ################################################################################################                
        # update the file_meta dictionary with the phone transcriptions and their corresponding end times
                file_meta[file_name_id]['phones'] = csv_input[0]
                file_meta[file_name_id]['end_times'] = csv_input[1]

    with open(folder_path + "/" + file_name_id +".csv", 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_input)


   
    # save the updated file_meta dictionary to a CSV file
    with open(main_path+'/DeepSpeech/data/file_meta_end_phones_'+Lan+'.csv', 'w', newline='') as csv_file:
  
        # Define the fieldnames of the CSV file
        fieldnames = list(file_meta.keys())

        # Create a writer object
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()

        # Write the data rows
        writer.writerow(file_meta)

------- IntervalTier Example -------
common_voice_de_28897371.TextGrid - export Phnes and End_times Done
------- IntervalTier Example -------
common_voice_de_27916918.TextGrid - export Phnes and End_times Done
------- IntervalTier Example -------
common_voice_de_30676740.TextGrid - export Phnes and End_times Done


In [23]:
file_meta["common_voice_de_30676740"]['phones']

['',
 'spn',
 'v',
 'ʊ',
 'ʁ',
 'd',
 'ə',
 'f',
 'ɔ',
 'n',
 'z',
 'aj',
 'n',
 'n̩',
 'spn',
 'ʊ',
 'n',
 't',
 'aj',
 'n',
 'ɐ',
 'tʰ',
 'a',
 'n',
 'tʰ',
 'ə',
 'ɛ',
 'ɐ',
 'ts',
 'oː',
 'ɡ',
 'n̩',
 '']

In [24]:
file_meta["common_voice_de_30676740"]['end_times'] 

[1.09,
 1.68,
 2.0,
 2.07,
 2.08,
 2.13,
 2.17,
 2.26,
 2.31,
 2.4,
 2.51,
 2.6,
 2.66,
 2.73,
 3.52,
 3.57,
 3.62,
 3.69,
 3.8,
 3.85,
 3.91,
 4.06,
 4.12,
 4.18,
 4.32,
 4.36,
 4.43,
 4.47,
 4.64,
 4.77,
 4.83,
 4.96,
 6.264]